In [ ]:
# Preflight diagnostics (non-blocking): network + GPU/PyTorch compatibility
import sys
import re
import subprocess
import importlib


def section(title):
    print()
    print("=" * 80)
    print(title)
    print("=" * 80)


def run_command_live(cmd, desc=None):
    if desc:
        print(f"[RUN] {desc}")
    print("[CMD]", " ".join(cmd))
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )
    lines = []
    if proc.stdout is not None:
        for line in proc.stdout:
            lines.append(line)
            print(line, end="")
    code = proc.wait()
    print(f"[EXIT CODE] {code}")
    return code, "".join(lines)


def ensure_speedtest_cli():
    if importlib.util.find_spec("speedtest") is not None:
        print("speedtest-cli module: already installed")
        return True

    print("speedtest-cli module: not found; attempting installation with live logs...")
    code, _ = run_command_live([sys.executable, "-m", "pip", "install", "-U", "speedtest-cli"], "Installing speedtest-cli")
    if code != 0:
        print("[WARN] Could not install speedtest-cli. Internet speed measurements will be skipped.")
        return False

    ok = importlib.util.find_spec("speedtest") is not None
    print("speedtest-cli install status:", "OK" if ok else "FAILED")
    return ok


def run_speedtest():
    results = {"download_mbps": None, "upload_mbps": None, "error": None}
    try:
        import speedtest

        print("Running speedtest-cli (this can take ~20-60s)...")
        st = speedtest.Speedtest(secure=True)
        print("- Getting best server...")
        st.get_best_server()
        print("- Testing download speed...")
        dl_bps = st.download()
        print("- Testing upload speed...")
        ul_bps = st.upload()

        results["download_mbps"] = dl_bps / 1_000_000
        results["upload_mbps"] = ul_bps / 1_000_000
    except Exception as e:
        results["error"] = str(e)
    return results


def avg_ping(host):
    # Uses Linux ping summary line: rtt min/avg/max/mdev = ...
    cmd = ["ping", "-c", "3", "-W", "2", host]
    print(f"\nPinging {host}...")
    code, out = run_command_live(cmd)
    if code != 0:
        return None, f"ping command failed with code {code}"

    m = re.search(r"=\s*([\d\.]+)/([\d\.]+)/([\d\.]+)/([\d\.]+)", out)
    if not m:
        return None, "unable to parse ping output"
    return float(m.group(2)), None


def parse_version_tuple(v):
    try:
        core = re.match(r"(\d+)\.(\d+)", str(v))
        if not core:
            return (0, 0)
        return (int(core.group(1)), int(core.group(2)))
    except Exception:
        return (0, 0)


def parse_semver_tuple(v):
    match = re.match(r"(\d+)\.(\d+)\.(\d+)", str(v))
    if not match:
        return (0, 0, 0)
    return tuple(int(match.group(i)) for i in range(1, 4))


def ensure_pillow_compatibility():
    try:
        from PIL import __version__ as pillow_version
    except Exception:
        print("Pillow version is compatible (below 12)")
        return

    if parse_semver_tuple(pillow_version) >= (12, 0, 0):
        code, _ = run_command_live([sys.executable, "-m", "pip", "install", "Pillow==11.1.0"], "Downgrading Pillow to 11.1.0")
        print("Pillow downgraded to 11.1.0")
    else:
        print("Pillow version is compatible (below 12)")


ensure_pillow_compatibility()

section("1) Internet connection checks (non-blocking)")
can_speedtest = ensure_speedtest_cli()
net = {"download_mbps": None, "upload_mbps": None, "civitai_ping_ms": None, "hf_ping_ms": None}

if can_speedtest:
    sp = run_speedtest()
    net["download_mbps"] = sp.get("download_mbps")
    net["upload_mbps"] = sp.get("upload_mbps")
    if sp.get("error"):
        print("[WARN] speedtest failed:", sp["error"])
else:
    print("[WARN] speedtest skipped because speedtest-cli is unavailable.")

civitai_ping, civitai_err = avg_ping("civitai.com")
hf_ping, hf_err = avg_ping("huggingface.co")
net["civitai_ping_ms"] = civitai_ping
net["hf_ping_ms"] = hf_ping

print("\n--- Network Results ---")
print(f"Download (Mbps): {net['download_mbps']:.2f}" if net['download_mbps'] is not None else "Download (Mbps): N/A")
print(f"Upload   (Mbps): {net['upload_mbps']:.2f}" if net['upload_mbps'] is not None else "Upload   (Mbps): N/A")
print(f"Ping civitai.com (ms avg): {net['civitai_ping_ms']:.2f}" if net['civitai_ping_ms'] is not None else f"Ping civitai.com (ms avg): N/A ({civitai_err})")
print(f"Ping huggingface.co (ms avg): {net['hf_ping_ms']:.2f}" if net['hf_ping_ms'] is not None else f"Ping huggingface.co (ms avg): N/A ({hf_err})")

net_ok = (
    net['download_mbps'] is not None and net['download_mbps'] >= 50
    and net['upload_mbps'] is not None and net['upload_mbps'] >= 100
    and net['civitai_ping_ms'] is not None and net['civitai_ping_ms'] < 100
    and net['hf_ping_ms'] is not None and net['hf_ping_ms'] < 100
)
print("Recommendation:")
if net_ok:
    print("- Network looks OK (download >=50 Mbps, upload >=100 Mbps, pings <100 ms).")
else:
    print("- Network is below suggested threshold or incomplete. Consider dropping instance if this impacts your workflow.")


section("2) GPU + PyTorch compatibility checks (non-blocking)")
try:
    import torch
    torch_import_ok = True
except Exception as e:
    torch_import_ok = False
    torch = None
    torch_import_err = str(e)

if not torch_import_ok:
    print("PyTorch import: FAILED")
    print("Error:", torch_import_err)
    print("Recommendation: PyTorch unavailable; consider dropping instance for compatibility reasons.")
else:
    gpu_available = bool(torch.cuda.is_available())
    gpu_name = None
    compute_cap = None
    arch_list = None
    pytorch_test = "Not run"
    pytorch_test_error = None

    print(f"PyTorch version: {torch.__version__}")
    print(f"PyTorch CUDA version: {torch.version.cuda}")
    print(f"GPU available: {'Yes' if gpu_available else 'No'}")

    if gpu_available:
        try:
            gpu_name = torch.cuda.get_device_name(0)
        except Exception as e:
            gpu_name = f"Unavailable ({e})"

        try:
            compute_cap = torch.cuda.get_device_capability(0)
        except Exception as e:
            compute_cap = f"Unavailable ({e})"

        try:
            arch_list = torch.cuda.get_arch_list()
        except Exception as e:
            arch_list = f"Unavailable ({e})"

        try:
            _ = torch.rand(1).cuda()
            pytorch_test = "Passed"
        except RuntimeError as e:
            pytorch_test = "Failed"
            pytorch_test_error = str(e)
        except Exception as e:
            pytorch_test = "Failed"
            pytorch_test_error = str(e)

    print(f"GPU name: {gpu_name}")
    print(f"Compute capability: {compute_cap}")
    print(f"Supported architectures (torch.cuda.get_arch_list): {arch_list}")
    if pytorch_test == "Passed":
        print("PyTorch CUDA test: Passed")
    elif pytorch_test == "Failed":
        print(f"PyTorch CUDA test: Failed: {pytorch_test_error}")
    else:
        print("PyTorch CUDA test: Not run (GPU unavailable)")

    allowed_caps = {(8, 0), (8, 6), (8, 9)}
    torch_ver = parse_version_tuple(torch.__version__)
    torch_ok = torch_ver >= (2, 4)
    cuda_ok = bool(torch.version.cuda and str(torch.version.cuda).startswith("12"))
    cap_ok = isinstance(compute_cap, tuple) and compute_cap in allowed_caps
    test_ok = pytorch_test == "Passed"

    print("Recommendation:")
    if gpu_available and cap_ok and torch_ok and cuda_ok and test_ok:
        print("- Compatibility looks OK (compute_cap in [(8,0),(8,6),(8,9)], PyTorch >=2.4, CUDA 12.x, test passed).")
    else:
        if pytorch_test_error and "no kernel image" in pytorch_test_error.lower():
            print("- Detected 'no kernel image' runtime issue. Likely torch/CUDA arch mismatch.")
            print("- Consider dropping this instance or switching to a build that supports your GPU (e.g., RTX 50xx may need cu128+ nightly).")
        else:
            print("- Potential compatibility issue detected (GPU/PyTorch/CUDA/test mismatch).")
            print("- Consider dropping for compatibility issues (e.g., RTX 50xx may need cu128+ nightly).")

print("\nDone: diagnostics completed without raising hard errors.")



<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
MAIN SETTINGS
</div>

In [ ]:
#MAIN

import os
import sys
import re
import time
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass
from urllib.parse import urlencode

#main vars
ON_VAST = any(k in os.environ for k in ("VAST_CONTAINERLABEL", "VAST_TCP_PORT_22", "CONTAINER_ID")) or os.path.exists('/workspace')
MAX_PARALLEL_DOWNLOADS = max(1, int(os.environ.get("MAX_PARALLEL_DOWNLOADS", "3")))
MIN_VALID_FILE_BYTES = int(os.environ.get("MIN_VALID_FILE_BYTES", "1000000"))

###Main Dependencies###
system_packages = [
    "aria2",
    "libjpeg-dev",
    "zlib1g-dev",
    "libpng-dev",
    "libfreetype6-dev",
    "libopenjp2-7-dev",
    "build-essential",
]
print("Checking/installing download and Pillow build dependencies...")
try:
    subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
    result = subprocess.run(["apt", "install", "-y", "-qq", *system_packages], capture_output=True, text=True)
    if result.returncode == 0:
        print("System dependencies are ready")
    else:
        print("Install failed (code {}):".format(result.returncode))
        print("stderr:", result.stderr.strip())
except subprocess.CalledProcessError as e:
    print(f"apt error (code {e.returncode}): {e.stderr}")
except Exception as e:
    print(f"Unexpected error: {e}")
if shutil.which("aria2c") is None:
    raise RuntimeError("aria2c is still unavailable after apt install")

###Determing directories###
possible_bases = [
    "/workspace",
]
BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break
if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)
print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")
VOLUME_DIR       = os.path.join(BASE_DIR, "volume")
GEN_DIR          = os.path.join(BASE_DIR, "gen")
IMAGES_DIR       = os.path.join(GEN_DIR, "Images")
for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR, VOLUME_DIR, GEN_DIR, IMAGES_DIR]:
    os.makedirs(d, exist_ok=True)

###Dependencies used by generation cell###
for pkg in ["openpyxl", "requests"]:
    try:
        __import__(pkg)
    except ImportError:
        print(f"Installing missing dependency: {pkg}")
        subprocess.run(["python3", "-m", "pip", "install", "-q", pkg], check=True)

###Token Authorization###
def get_secret(name: str):
    """Get secret from environment variables only."""
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"
    return None, None
CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")
if not CIVITAI_TOKEN:
    manual_civitai = getpass("Enter CIVITAI_TOKEN (leave blank to skip): ").strip()
    if manual_civitai:
        CIVITAI_TOKEN, CIVITAI_SRC = manual_civitai, "manual_input"
if not HF_TOKEN:
    manual_hf = getpass("Enter HF_TOKEN (leave blank to skip): ").strip()
    if manual_hf:
        HF_TOKEN, HF_SRC = manual_hf, "manual_input"
TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN
print("Token sources:")
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")
if ON_VAST:
    print("Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.")
if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")
if not TOKENS:
    raise RuntimeError("No tokens were provided. Set secrets or enter at least one token (CivitAI or HF).")

###Download functions###
def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url

def _looks_valid_file(path, min_bytes=MIN_VALID_FILE_BYTES):
    return os.path.exists(path) and os.path.getsize(path) > min_bytes

def _human_mb(num_bytes):
    return f"{num_bytes / (1024 * 1024):.1f} MB"

def _estimate_expected_mb(label):
    # Examples: "151 MB", "6,46 GB"
    match = re.search(r"(\d+[\.,]?\d*)\s*(MB|GB)", label, re.IGNORECASE)
    if not match:
        return None
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2).upper()
    return value * (1024 if unit == "GB" else 1)

def _size_sanity_warning(path, expected_mb, tolerance=0.7):
    if expected_mb is None or not os.path.exists(path):
        return
    actual_mb = os.path.getsize(path) / (1024 * 1024)
    if actual_mb < expected_mb * tolerance:
        print(f"  WARNING: file size looks low ({actual_mb:.1f} MB vs expected ~{expected_mb:.1f} MB)")

def _has_min_free_disk(path, required_mb, reserve_mb=1024):
    if required_mb is None:
        return True
    usage = shutil.disk_usage(path)
    free_mb = usage.free / (1024 * 1024)
    return free_mb >= (required_mb + reserve_mb)



def _run_command_with_live_output(cmd, prefix=""):
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    return proc.returncode, proc.stdout or ""

def _download_one(job, target_dir):
    label, url, filename, token_name = job
    token = TOKENS.get(token_name)
    output_path = os.path.join(target_dir, filename)
    expected_mb = _estimate_expected_mb(label)
    if _looks_valid_file(output_path):
        _size_sanity_warning(output_path, expected_mb)
        return (filename, "skipped", "exists")
    if expected_mb is not None and not _has_min_free_disk(target_dir, expected_mb):
        return (filename, "error", "insufficient disk")
    tmp_path = output_path + ".part"
    if os.path.exists(tmp_path):
        os.remove(tmp_path)
    final_url = _prepare_download_url(url, token if token_name == "CIVITAI" else None)
    # CivitAI signed links on Vast can return 403 with aggressive multi-connection mode.
    max_conn = "1" if token_name == "CIVITAI" else "16"
    split = "1" if token_name == "CIVITAI" else "16"
    cmd = [
        "aria2c",
        "--allow-overwrite=true",
        "--auto-file-renaming=false",
        "--continue=true",
        f"--max-connection-per-server={max_conn}",
        f"--split={split}",
        "--min-split-size=1M",
        "--console-log-level=warn",
        "--summary-interval=1",
        "--check-certificate=false",
        "--header=User-Agent: Mozilla/5.0",
        "--header=Referer: https://civitai.com/",
        "--out", os.path.basename(tmp_path),
        "--dir", target_dir,
        final_url,
    ]
    if token_name == "HF_TOKEN" and token:
        cmd.insert(-1, f"--header=Authorization: Bearer {token}")
    started = time.time()
    code, output = _run_command_with_live_output(cmd, prefix=f"[{label}] ")
    if code != 0:
        msg = (output or "").strip() or f"aria2c exited {code}"
        # Retry once for CivitAI with curl fallback if aria2 still gets 403.
        if token_name == "CIVITAI":
            curl_cmd = [
                "curl", "-L", "--fail", "--retry", "2", "--retry-delay", "3",
                "-A", "Mozilla/5.0", "-e", "https://civitai.com/",
                "-o", tmp_path, final_url
            ]
            curl_code, curl_output = _run_command_with_live_output(curl_cmd, prefix=f"[{label}][curl] ")
            if curl_code == 0 and _looks_valid_file(tmp_path):
                os.replace(tmp_path, output_path)
                _size_sanity_warning(output_path, expected_mb)
                elapsed = max(0.001, time.time() - started)
                size_bytes = os.path.getsize(output_path)
                speed_mb_s = (size_bytes / (1024 * 1024)) / elapsed
                return (filename, "downloaded", f"{_human_mb(size_bytes)} in {elapsed:.1f}s ({speed_mb_s:.2f} MB/s)")
            curl_err = (curl_output or "curl failed").strip()
            msg = f"{msg}\nFallback curl: {curl_err}"
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (filename, "error", msg)
    if not _looks_valid_file(tmp_path):
        size = os.path.getsize(tmp_path) if os.path.exists(tmp_path) else 0
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (filename, "error", f"downloaded file too small ({_human_mb(size)})")
    os.replace(tmp_path, output_path)
    _size_sanity_warning(output_path, expected_mb)
    elapsed = max(0.001, time.time() - started)
    size_bytes = os.path.getsize(output_path)
    speed_mb_s = (size_bytes / (1024 * 1024)) / elapsed
    return (filename, "downloaded", f"{_human_mb(size_bytes)} in {elapsed:.1f}s ({speed_mb_s:.2f} MB/s)")

def run_download_list(download_list, target_dir, title):
    print(f"\n=== {title} ===")
    os.makedirs(target_dir, exist_ok=True)
    if not download_list:
        print("No items.")
        return
    workers = min(MAX_PARALLEL_DOWNLOADS, len(download_list))
    print(f"Parallel downloads: {workers}")
    ok = 0
    fail = 0
    with ThreadPoolExecutor(max_workers=workers) as ex:
        futures = [ex.submit(_download_one, job, target_dir) for job in download_list]
        for fut in as_completed(futures):
            filename, status, info = fut.result()
            if status == "downloaded":
                ok += 1
                print(f"**DOWNLOADED:** {filename}")
            elif status == "skipped":
                ok += 1
                print(f"**SKIPPED:** {filename}")
            else:
                fail += 1
                print(f"**ERROR:** {filename} -> {info}")
    print(f"Done: OK={ok}, FAIL={fail}")
    if fail > 0:
        raise RuntimeError(f"Some downloads failed in {title}: {fail} item(s)")

In [ ]:
### FORGE BOOTSTRAP ###

import os
import shutil
import subprocess

#Vars
required_packages = ["git", "python3-venv", "python3-pip", "libjpeg-dev", "zlib1g-dev", "libpng-dev", "build-essential"]
print("Checking/installing platform dependencies...")
subprocess.run(["apt", "update", "-qq"], check=False)
subprocess.run(["apt", "install", "-y", "-qq", *required_packages], check=False)
launch_script = os.path.join(FORGE_DIR, "webui.sh")
git_head = os.path.join(FORGE_DIR, ".git", "HEAD")
forge_ready = os.path.isfile(launch_script) and os.path.isfile(git_head)

#Forge install
if forge_ready:
    print("WebUI Forge already exists and looks valid, skipping clone.")
else:
    if os.path.isdir(FORGE_DIR):
        print("FORGE_DIR exists but WebUI Forge is incomplete/corrupted. Recreating...")
        shutil.rmtree(FORGE_DIR)
    print("Cloning WebUI Forge...")
    subprocess.run([
        "git", "clone", "https://github.com/lllyasviel/stable-diffusion-webui-forge", FORGE_DIR
    ], check=True)
    if not os.path.isfile(os.path.join(FORGE_DIR, "webui.sh")):
        raise FileNotFoundError("Clone completed but webui.sh not found. Check repository state.")
print("Optional bootstrap finished.")

In [ ]:
###MAIN DOWNLOAD CELL###
# CONTROLNET INSTALL OPTIONS
# Options: A=force clean reinstall; B="update" for git pull in existing repo; C="skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()

if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")
print(f"ControlNet extension path: {CONTROLNET_DIR}")
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")

if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)
    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)

os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)
print("ControlNet repository is ready")
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

#ControlNET models download
controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
]
run_download_list(controlnet_models_to_download, CONTROLNET_MODELS_DIR, "ControlNet")

#Models(checkpoints) download
models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "wai_v160.safetensors", "CIVITAI"),
]
run_download_list(models_to_download, MODELS_DIR, "Checkpoints")

#LoRa download
lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_v35.safetensors", "CIVITAI"),
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"),
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_styles.safetensors", "CIVITAI"),
    ("Pixel Art Style IL V7 435 MB", "https://civitai.com/api/download/models/2661972?type=Model&format=SafeTensor", "pixel_art.safetensors", "CIVITAI"),
]
run_download_list(lora_to_download, LORA_DIR, "LoRA")

In [ ]:
### RUN WEBUI ###

import os
import re
import time
import subprocess
from pathlib import Path

forge_dir = Path(FORGE_DIR)
if not forge_dir.exists():
    raise FileNotFoundError(f"FORGE_DIR не найден: {forge_dir}")

launch_utils_path = forge_dir / "modules" / "launch_utils.py"

# Force Pillow pin in Forge requirement files (Forge may reinstall dependencies on startup)
for req_name in ("requirements.txt", "requirements_versions.txt"):
    req_path = forge_dir / req_name
    if not req_path.exists():
        continue

    lines = req_path.read_text(encoding="utf-8").splitlines()
    updated = []
    pillow_line_found = False

    for line in lines:
        if line.strip().lower().startswith("pillow"):
            updated.append("pillow<12")
            pillow_line_found = True
        else:
            updated.append(line)

    if not pillow_line_found:
        updated.append("pillow<12")

    req_path.write_text("\n".join(updated) + "\n", encoding="utf-8")
    print(f"Pinned Pillow in {req_path}")

if launch_utils_path.exists():
    content = launch_utils_path.read_text(encoding="utf-8")

    clip_line_pattern = re.compile(
        r'^(?P<indent>\s*)run_pip\(f"install(?: --no-build-isolation)?(?: --no-use-pep517)? \{clip_package\}", "clip"\)\s*$',
        re.MULTILINE,
    )
    def _replace_clip_install(match):
        indent = match.group("indent")
        return (
            f'{indent}run_pip("install setuptools==69.5.1 wheel", "clip build deps")\n'
            f'{indent}run_pip(f"install --no-build-isolation {{clip_package}}", "clip")'
        )
    content_new, replacements = clip_line_pattern.subn(_replace_clip_install, content, count=1)
    numpy_fix_pattern = re.compile(
        r'^(?P<indent>\s*)startup_timer\.record\("install torch"\)\s*$',
        re.MULTILINE,
    )
    def _replace_torch_record(match):
        indent = match.group("indent")
        return (
            f"{indent}run_pip('install \"numpy==2.2.6\" \"scikit-image>=0.24.0\" --force-reinstall', 'numpy/skimage compatibility')\n"
            f'{indent}startup_timer.record("install torch")'
        )
    torch_fix_replacements = 0
    if "numpy/skimage compatibility" not in content_new:
        content_new, torch_fix_replacements = numpy_fix_pattern.subn(_replace_torch_record, content_new, count=1)
        if torch_fix_replacements:
            print("Patched torch stage (reinstalled numpy 2.x + scikit-image for ABI compatibility).")
        else:
            print("Torch-stage compatibility patch not applied (marker not found).")
    late_numpy_pattern = re.compile(r'^(?P<indent>\s*)import webui(?:\s*#.*)?\s*$', re.MULTILINE)
    def _replace_import_webui(match):
        indent = match.group("indent")
        return (
            f"{indent}run_pip('install \"numpy==2.2.6\" \"scikit-image>=0.24.0\" --force-reinstall', 'numpy/skimage late compatibility')\n"
            f'{indent}import webui'
        )
    late_fix_replacements = 0
    if "numpy/skimage late compatibility" not in content_new:
        content_new, late_fix_replacements = late_numpy_pattern.subn(_replace_import_webui, content_new, count=1)
        if late_fix_replacements:
            print("Patched start stage (re-pin numpy/scikit-image before importing webui).")
        else:
            print("Start-stage compatibility patch not applied (import webui line not found).")
    avif_fix_replacements = 0
    avif_fix_pattern = re.compile(r'^(?P<indent>\s*)import webui(?:\s*#.*)?\s*$', re.MULTILINE)
    def _replace_import_webui_avif(match):
        indent = match.group("indent")
        return (
            f"{indent}run_pip('install \"pillow<12\" --force-reinstall', 'pillow late compatibility')\n"
            f"{indent}run_pip('install pillow-avif-plugin', 'pillow_avif plugin')\n"
            f'{indent}import webui'
        )
    if "pillow_avif plugin" not in content_new:
        content_new, avif_fix_replacements = avif_fix_pattern.subn(_replace_import_webui_avif, content_new, count=1)
        if avif_fix_replacements:
            print("Patched start stage (ensured Pillow<12 and pillow-avif-plugin before importing webui).")
        else:
            print("Start-stage Pillow/AVIF patch not applied (import webui line not found).")
    if replacements or torch_fix_replacements or late_fix_replacements or avif_fix_replacements:
        launch_utils_path.write_text(content_new, encoding="utf-8")
    if replacements:
        print("Patched CLIP install command (fixed flags + preserved indentation).")
    else:
        print("CLIP install patch already applied or target line not found.")
else:
    print(f"Warning: {launch_utils_path} not found, skipping CLIP patch.")

# Workaround for frequent startup issue in ControlNet: soft_inpainting.py

soft_inpainting_path = forge_dir / "extensions" / "sd-webui-controlnet" / "scripts" / "soft_inpainting.py"


soft_inpainting_disabled_path = soft_inpainting_path.with_suffix(".py.disabled")


if soft_inpainting_path.exists():
    os.replace(soft_inpainting_path, soft_inpainting_disabled_path)
    print(f"Disabled problematic script: {soft_inpainting_path.name} -> {soft_inpainting_disabled_path.name}")
else:
    print("soft_inpainting.py already disabled or not present.")

cmd = ["bash", "webui.sh", "-f", "--xformers", "--api", "--port", "17860"]
run_env = os.environ.copy()
run_env["MPLBACKEND"] = "Agg"
print("Running (background):", " ".join(cmd), "in", forge_dir)
print("MPLBACKEND forced to:", run_env["MPLBACKEND"])
venv_python = forge_dir / "venv" / "bin" / "python"

if venv_python.exists():
    print("Ensuring Forge venv has Pillow < 12 + AVIF plugin")
    subprocess.run([str(venv_python), "-m", "pip", "install", "pillow<12", "--force-reinstall"], check=False)
    subprocess.run([str(venv_python), "-m", "pip", "install", "pillow-avif-plugin"], check=False)

    print("Installing missing dependency in Forge venv: joblib")
    subprocess.run([str(venv_python), "-m", "pip", "install", "joblib"], check=False)

    print("Trying optional dependency in Forge venv: insightface")
    insightface_result = subprocess.run([str(venv_python), "-m", "pip", "install", "insightface"], check=False)
    if insightface_result.returncode != 0:
        print("Warning: insightface installation failed (optional dependency).")
else:
    print(f"Warning: venv python not found at {venv_python}, skipping pillow/joblib/insightface install.")

pid_path = forge_dir / "webui.pid"
log_path = forge_dir / "webui.log"

def _pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False

def _log_tail(chars: int = 4000) -> str:
    if not log_path.exists():
        return "(webui.log not found yet)"
    return log_path.read_text(encoding="utf-8", errors="ignore")[-chars:]

running_pid = None

if pid_path.exists():
    try:
        old_pid = int(pid_path.read_text(encoding="utf-8").strip())
        if _pid_alive(old_pid):
            running_pid = old_pid
            print(f"WebUI already running with PID={old_pid}. Можно запускать последнюю ячейку.")
        else:
            pid_path.unlink(missing_ok=True)
    except Exception:
        pid_path.unlink(missing_ok=True)

started_proc = None

if running_pid is None:
    with open(log_path, "a", encoding="utf-8") as log_f:
        started_proc = subprocess.Popen(
            cmd,
            cwd=forge_dir,
            env=run_env,
            stdout=log_f,
            stderr=subprocess.STDOUT,
            start_new_session=True,
            text=True,
        )
    pid_path.write_text(str(started_proc.pid), encoding="utf-8")
    running_pid = started_proc.pid
    print(f"WebUI started in background. PID={running_pid}")
    print(f"Logs: {log_path}")

observe_timeout = int(os.environ.get("FORGE_STARTUP_OBSERVE_TIMEOUT", "120"))
observe_interval = float(os.environ.get("FORGE_STARTUP_OBSERVE_INTERVAL", "2"))

for _ in range(max(1, int(observe_timeout / observe_interval))):
    tail = _log_tail()
    if started_proc is not None:
        rc = started_proc.poll()
        if rc is not None:
            raise RuntimeError(
                f"WebUI process exited early with code {rc}.\n"
                f"Check logs at: {log_path}\n"
                f"Last log lines:\n{tail}"
            )
    if "Running on local URL" in tail or "Uvicorn running" in tail:
        print("WebUI appears ready. Можно запускать последнюю ячейку.")
        break

    time.sleep(observe_interval)
else:
    print("WebUI всё ещё запускается в фоне. Запускайте последнюю ячейку: она дождётся API.")
    print(f"Если API не поднимется, проверьте лог: {log_path}")

In [ ]:
### API GENERATION FROM XLSX ###

import os
import re
import json
import glob
import time
import shutil
import zipfile
import base64
import subprocess
from datetime import datetime
import requests

try:
    from openpyxl import load_workbook
except ImportError:
    import subprocess, sys
    print("Installing missing dependency: openpyxl")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'openpyxl'], check=True)
    from openpyxl import load_workbook


API_BASE = os.environ.get("FORGE_API_BASE", "http://127.0.0.1:17860")
TXT2IMG_URL = f"{API_BASE}/sdapi/v1/txt2img"
OPENAPI_URL = f"{API_BASE}/openapi.json"
PROMPTS_CANDIDATES = [
    os.path.join(GEN_DIR, "prompts.xlsx"),
    os.path.join(GEN_DIR, "prompts.xlxs"),
    os.path.join(GEN_DIR, "Prompts.xlsx"),
    os.path.join(GEN_DIR, "Prompts.xlxs"),
    os.path.join(BASE_DIR, "prompts.xlsx"),
    os.path.join(BASE_DIR, "prompts.xlxs"),
    os.path.join(BASE_DIR, "Prompts.xlsx"),
    os.path.join(BASE_DIR, "Prompts.xlxs"),
    "/workspace/gen/prompts.xlsx",
    "/workspace/gen/prompts.xlxs",
]
PROMPTS_CANDIDATES = list(dict.fromkeys(PROMPTS_CANDIDATES))
API_READY_TIMEOUT = int(os.environ.get("FORGE_API_READY_TIMEOUT", "600"))
API_READY_INTERVAL = float(os.environ.get("FORGE_API_READY_INTERVAL", "3"))

def wait_for_api_ready(base_url: str, timeout_s: int = API_READY_TIMEOUT, interval_s: float = API_READY_INTERVAL):
    start = time.time()
    last_error = None
    webui_log = os.path.join(FORGE_DIR, "webui.log")
    webui_pid = os.path.join(FORGE_DIR, "webui.pid")

    def _log_tail(chars: int = 4000):
        if not os.path.exists(webui_log):
            return "(webui.log not found yet)"
        with open(webui_log, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()[-chars:]

    def _is_pid_alive():
        try:
            if not os.path.exists(webui_pid):
                return None
            with open(webui_pid, "r", encoding="utf-8") as f:
                pid = int(f.read().strip())
            os.kill(pid, 0)
            return pid
        except Exception:
            return False

    while (time.time() - start) < timeout_s:
        try:
            r = requests.get(f"{base_url}/sdapi/v1/progress", timeout=10)
            if r.status_code < 500:
                return True
            last_error = f"HTTP {r.status_code}"
        except Exception as e:
            last_error = str(e)

        pid_state = _is_pid_alive()
        if pid_state is False:
            raise RuntimeError(
                "Forge API недоступен и процесс WebUI уже завершился.\n"
                f"Last error: {last_error}\n"
                f"WebUI log tail:\n{_log_tail()}"
            )

        time.sleep(interval_s)

    raise RuntimeError(
        f"Forge API is not reachable at {base_url} after {timeout_s}s. "
        f"Last error: {last_error}.\n"
        f"WebUI log tail:\n{_log_tail()}\n"
        "Run the Forge startup cell first. If this is first launch, increase FORGE_API_READY_TIMEOUT (e.g. 900)."
    )

LOG_PATH = os.path.join(VOLUME_DIR, "log.txt")
API_IMAGES_DIR = os.path.join(OUTPUTS_DIR, "api_generated")
REQUEST_DUMPS_DIR = os.path.join(VOLUME_DIR, "request_dumps")

os.makedirs(API_IMAGES_DIR, exist_ok=True)
os.makedirs(REQUEST_DUMPS_DIR, exist_ok=True)

INPUT_IMAGE_DIRS = [
    os.path.join(GEN_DIR, "Images"),
    os.path.join(GEN_DIR, "images"),
]
INPUT_IMAGE_DIRS = [p for i, p in enumerate(INPUT_IMAGE_DIRS) if p and p not in INPUT_IMAGE_DIRS[:i]]
CONFLICT_RULES = [
    (("hr_resize_x", "hr_resize_y"), ("hr_scale",)),
]
CONTROLNET_FIELDS = [
    "enabled", "image", "mask", "weight", "module", "model", "resize_mode", "lowvram",
    "processor_res", "threshold_a", "threshold_b", "guidance_start", "guidance_end",
    "control_mode", "pixel_perfect",
]

def log_line(text: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {text}"
    print(line)
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")

def normalize_value(value):
    if isinstance(value, str):
        stripped = value.strip()
        lowered = stripped.lower()
        if lowered in {"", "null", "none", "nan"}:
            return None
        if lowered in {"true", "yes", "1"}:
            return True
        if lowered in {"false", "no", "0"}:
            return False
        if re.fullmatch(r"-?\d+", stripped):
            return int(stripped)
        if re.fullmatch(r"-?\d+\.\d+", stripped):
            return float(stripped)
        return stripped
    return value

def first_existing_path(candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

def parse_workbook(path):
    wb = load_workbook(path, data_only=True)
    ws = wb.active
    row1 = [normalize_value(v) for v in next(ws.iter_rows(min_row=1, max_row=1, values_only=True))]
    row2 = [normalize_value(v) for v in next(ws.iter_rows(min_row=2, max_row=2, values_only=True))]
    row1_headers_like = any(isinstance(v, str) and v in {"prompt", "negative_prompt", "sampler_name", "steps", "cn1_enabled"} for v in row1)
    if row1_headers_like:
        instruction = "{prompt}"
        headers = [str(v).strip() if v is not None else "" for v in row1]
        data_start_row = 2
    else:
        instruction_parts = [str(v).strip() for v in row1 if v is not None and str(v).strip()]
        if not instruction_parts:
            raise ValueError("The first line (instruction) is empty")
        instruction = " ".join(instruction_parts)
        headers = [str(v).strip() if v is not None else "" for v in row2]
        data_start_row = 3
    if not any(headers):
        raise ValueError("Table headers with variables not found")
    rows = []
    for row_idx in range(data_start_row, ws.max_row + 1):
        row_values = [normalize_value(v) for v in next(ws.iter_rows(min_row=row_idx, max_row=row_idx, values_only=True))]
        if all(v is None for v in row_values):
            continue
        row_map = {}
        for idx, key in enumerate(headers):
            if not key or key.startswith("S_"):
                continue
            value = row_values[idx] if idx < len(row_values) else None
            if value is None:
                continue
            row_map[key] = value
        rows.append(row_map)
    if not rows:
        raise ValueError("No data rows found")
    return instruction, rows

def render_instruction(template: str, variables: dict):
    def repl(match):
        key = match.group(1)
        return str(variables.get(key, match.group(0)))
    return re.sub(r"\{([a-zA-Z0-9_]+)\}", repl, template)

def apply_conflict_rules(payload: dict):
    cleaned = dict(payload)
    for primary_keys, conflicting_keys in CONFLICT_RULES:
        primary_present = all((k in cleaned and cleaned[k] is not None) for k in primary_keys)
        if primary_present:
            for ck in conflicting_keys:
                cleaned.pop(ck, None)
    return cleaned

def fetch_txt2img_params_dump():
    try:
        response = requests.get(OPENAPI_URL, timeout=30)
        response.raise_for_status()
        spec = response.json()
        schemas = spec.get("components", {}).get("schemas", {})
        candidates = [
            "StableDiffusionTxt2ImgProcessingApi",
            "Txt2ImgRequest",
            "StableDiffusionProcessingTxt2Img",
        ]
        for name in candidates:
            if name in schemas:
                props = schemas[name].get("properties", {})
                return json.dumps({k: v.get("type", "unknown") for k, v in props.items()}, ensure_ascii=False, indent=2)
        return json.dumps(spec.get("paths", {}).get("/sdapi/v1/txt2img", {}), ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Failed to get list of variables: {e}"

def resolve_image_path(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref
    ref = str(image_ref).strip()
    if not ref:
        return None
    if os.path.isabs(ref) and os.path.exists(ref):
        return ref
    candidates = [ref]
    if not os.path.splitext(ref)[1]:
        candidates += [f"{ref}.png", f"{ref}.jpg", f"{ref}.jpeg", f"{ref}.webp"]
    for base_dir in INPUT_IMAGE_DIRS:
        if not os.path.isdir(base_dir):
            continue
        for candidate in candidates:
            full = os.path.join(base_dir, candidate)
            if os.path.exists(full):
                return full
    for pattern in [f"**/{ref}", f"**/{ref}.*"]:
        for base_dir in INPUT_IMAGE_DIRS:
            if not os.path.isdir(base_dir):
                continue
            matches = glob.glob(os.path.join(base_dir, pattern), recursive=True)
            if matches:
                return matches[0]
    return None

def image_to_base64(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref
    image_path = resolve_image_path(image_ref)
    if not image_path:
        raise FileNotFoundError(f"Image/Mask not found: {image_ref}. Search dirs: {INPUT_IMAGE_DIRS}")
    ext = os.path.splitext(image_path)[1].lower()
    mime = {
        ".png": "image/png",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".webp": "image/webp",
        ".bmp": "image/bmp",
    }.get(ext, "application/octet-stream")
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

def parse_override_settings(value):
    if value in (None, "", {}):
        return {}
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        txt = value.strip()
        if not txt:
            return {}
        if txt.startswith("{"):
            return json.loads(txt)
        return json.loads("{" + txt + "}")
    raise ValueError(f"Unsupported override_settings format: {type(value)}")

def deep_clean(value):
    if isinstance(value, dict):
        cleaned = {}
        for k, v in value.items():
            cv = deep_clean(v)
            if cv is None:
                continue
            if isinstance(cv, str) and not cv.strip():
                continue
            if isinstance(cv, dict) and not cv:
                continue
            cleaned[k] = cv
        return cleaned
    if isinstance(value, list):
        cleaned_list = []
        for item in value:
            ci = deep_clean(item)
            if ci is None:
                continue
            if isinstance(ci, str) and not ci.strip():
                continue
            if isinstance(ci, dict) and not ci:
                continue
            cleaned_list.append(ci)
        return cleaned_list
    if isinstance(value, str):
        stripped = value.strip()
        if stripped == "[]":
            return []
        if stripped.lower() in {"", "null", "none", "nan"}:
            return None
        return stripped
    return value

def dump_payload(payload, generation_idx):
    dump_path = os.path.join(REQUEST_DUMPS_DIR, f"request_{generation_idx:04d}.json")
    with open(dump_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return dump_path

def build_payload(row_values, instruction_template):
    payload = dict(row_values)
    if "prompt" not in payload:
        payload["prompt"] = render_instruction(instruction_template, row_values)
    payload = {k: v for k, v in payload.items() if v is not None}
    if "override_settings" in payload:
        payload["override_settings"] = parse_override_settings(payload.get("override_settings"))
    controlnet_args = []
    for idx in (1, 2, 3):
        unit = {}
        for field in CONTROLNET_FIELDS:
            key = f"cn{idx}_{field}"
            if key in payload:
                unit[field] = payload.pop(key)
        if not unit:
            continue
        if "enabled" not in unit:
            unit["enabled"] = True
        if unit.get("image") is not None:
            unit["image"] = image_to_base64(unit.get("image"))
        if unit.get("mask") is not None:
            unit["mask"] = image_to_base64(unit.get("mask"))
        unit = deep_clean(unit)
        if unit.get("enabled"):
            controlnet_args.append(unit)
    if controlnet_args:
        payload.setdefault("alwayson_scripts", {})
        payload["alwayson_scripts"]["ControlNet"] = {"args": controlnet_args}
    payload = deep_clean(payload)
    payload = apply_conflict_rules(payload)
    payload = deep_clean(payload)
    return payload

def expected_image_count(payload):
    batch_size = int(payload.get("batch_size", 1) or 1)
    n_iter = int(payload.get("n_iter", 1) or 1)
    return max(1, batch_size) * max(1, n_iter)

def save_images_from_response(images_b64, generation_idx):
    saved = 0
    for i, b64 in enumerate(images_b64, start=1):
        image_data = b64.split(",", 1)[-1]
        file_name = f"gen_{generation_idx:04d}_{i:02d}.png"
        file_path = os.path.join(API_IMAGES_DIR, file_name)
        with open(file_path, "wb") as f:
            f.write(base64.b64decode(image_data))
        saved += 1
    return saved

def archive_outputs(tag: str):
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = os.path.join(VOLUME_DIR, f"outputs_{tag}_{stamp}.zip")
    with zipfile.ZipFile(archive_name, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTPUTS_DIR):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, OUTPUTS_DIR)
                zf.write(full_path, rel_path)
    for entry in os.listdir(OUTPUTS_DIR):
        p = os.path.join(OUTPUTS_DIR, entry)
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
    os.makedirs(API_IMAGES_DIR, exist_ok=True)
    log_line(f"ARCHIVE created: {archive_name}. OUTPUTS_DIR cleaned.")

wait_for_api_ready(API_BASE)

try:
    result = subprocess.run(["ss", "-tuln"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("Open TCP ports (ss):\n")
        print(result.stdout)
    else:
        print("ss didn't work")
except:
    print("ss not available")
prompts_path = first_existing_path(PROMPTS_CANDIDATES)
if not prompts_path:
    raise FileNotFoundError(
        "The file prompts.xlsx/prompts.xlxs was not found. Expected paths: " + ", ".join(PROMPTS_CANDIDATES)
    )

instruction_template, generation_rows = parse_workbook(prompts_path)
log_line(f"Loaded prompts file: {prompts_path}. Rows for generation: {len(generation_rows)}")
log_line(f"Image search dirs: {INPUT_IMAGE_DIRS}")
syntax_error_dumped = False
images_since_archive = 0
total_expected_images = 0
total_saved_images = 0

for generation_idx, row_values in enumerate(generation_rows, start=1):
    try:
        payload = build_payload(row_values, instruction_template)
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: payload build error: {e}")
        continue
    planned_images = expected_image_count(payload)
    total_expected_images += planned_images
    payload_dump_path = dump_payload(payload, generation_idx)
    log_line(f"#{generation_idx} request dump: {payload_dump_path}")
    if "alwayson_scripts" not in payload or not payload.get("alwayson_scripts", {}).get("ControlNet", {}).get("args"):
        log_line(f"#{generation_idx} INFO: no ControlNet image/mask attached in payload")
    try:
        response = requests.post(TXT2IMG_URL, json=payload, timeout=1800)
        if response.status_code != 200:
            detail = None
            try:
                detail = (response.json() or {}).get("detail")
            except Exception:
                detail = None
            err_text = str(detail or response.text or "")[:1200]
            if not syntax_error_dumped:
                params_dump = fetch_txt2img_params_dump()
                log_line(f"#{generation_idx} FAIL syntax/validation: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
                log_line("AVAILABLE PARAMS DUMP START")
                with open(LOG_PATH, "a", encoding="utf-8") as f:
                    f.write(params_dump + "\n")
                log_line("AVAILABLE PARAMS DUMP END")
                syntax_error_dumped = True
            else:
                log_line(f"#{generation_idx} FAIL: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
            continue
        saved_now = planned_images
        total_saved_images += saved_now
        images_since_archive += saved_now
        log_line(f"#{generation_idx} OK images={saved_now} expected={planned_images}")
        if images_since_archive >= 15:
            archive_outputs(tag=f"part_{generation_idx:04d}")
            images_since_archive = 0
    except requests.exceptions.Timeout:
        log_line(f"#{generation_idx} FAIL: timeout (possible heavy generation or API freeze)")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except requests.exceptions.RequestException as e:
        reason = str(e)
        if "out of memory" in reason.lower() or "oom" in reason.lower():
            reason = "OOM"
        log_line(f"#{generation_idx} FAIL: {reason}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: unexpected error: {e}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
if images_since_archive > 0:
    archive_outputs(tag="final")

log_line(f"Generation cycle completed. Requests={len(generation_rows)} expected_images={total_expected_images} saved_images={total_saved_images}")

In [ ]:
###Ports list###
import subprocess

# Пробуем ss (чаще всего работает в Ubuntu-контейнерах Vast.ai)
try:
    result = subprocess.run(["ss", "-tuln"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print("Открытые TCP-порты (ss):\n")
        print(result.stdout)
    else:
        print("ss не сработал")
except:
    print("ss недоступен")

In [ ]:
###Get a list of current settings###
import os
import requests

url = "http://127.0.0.1:17860"
options = requests.get(f'{url}/sdapi/v1/options').json()
models = requests.get(f'{url}/sdapi/v1/sd-models').json()

info_lines = [
    "OPTIONS KEYS:",
    str(list(options.keys())),
    "",
    "MODELS:",
    str(models),
]

for line in info_lines:
    print(line)

os.makedirs("/workspace/volume", exist_ok=True)
info_path = "/workspace/volume/models_and_deps_info.txt"
with open(info_path, "w", encoding="utf-8") as f:
    f.write("\n".join(info_lines) + "\n")
print(f"Saved info to {info_path}")
